In [2]:
from oil.lazy.lazy_matrix import LazyMatrix, lazy
from oil.lazy.low_rank import LowRank
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

In [4]:
float_type = np.float64
## Toy Problem 2 for stochastic eigen decomposition
## PCA
mb = 1000
rank = 100
n = int(1e6)
toy_eigs2 = np.sqrt(.9*np.ones(rank))
toy_eigs2[0] = 1
toy_eigs2[:10] = np.array([1.1,.95,.7,.5,.3,.01,.001,-.4,-.7,-.94]).astype(np.float32)
S = np.diag(toy_eigs2)
#U = torch.ones(rank,1e6).normal_().numpy()
U = np.random.randn(n,rank).astype(float_type)
U,_ = np.linalg.qr(U)
V = np.random.randn(rank,rank).astype(float_type)
V,_ = np.linalg.qr(V)
#V = V.astype(np.float64)
X = (np.sqrt(n)*U@S@V.T).astype(float_type)
cov = X.T@X/n

index = toy_eigs2.argmax()
v = V.T[index]

def toy_A():
    indices = np.random.permutation(n).reshape(n//mb,mb)
    for batch_ids in indices:
        m = len(batch_ids)
        x = X[batch_ids]
        yield [lambda v: (1/m)*x.T@(x@v)]

In [5]:
m = 1000
rank = 10
n = int(1e5)
toy_eigs2 = np.sqrt(.9*np.ones(rank))
toy_eigs2[0] = 1
toy_eigs2[:10] = np.array([1.1,.95,.7,.5,.3,.01,.001,-.4,-.7,-.94]).astype(np.float32)
S = np.diag(toy_eigs2)
#U = torch.ones(rank,1e6).normal_().numpy()
U = np.random.randn(n,rank).astype(float_type)
U,_ = np.linalg.qr(U)
V = np.random.randn(m,rank).astype(float_type)
V,_ = np.linalg.qr(V)
#V = V.astype(np.float64)
X = (np.sqrt(n)*U@S@V.T).astype(float_type)

In [6]:
X.shape

(100000, 1000)

In [7]:
U.shape

(100000, 10)

In [8]:
V.shape

(1000, 10)

In [9]:
X_lazy = LowRank(U@S,V)

In [10]:
X

array([[ 0.05499907,  0.02862312,  0.04330404, ..., -0.01993921,
        -0.01280142, -0.0457409 ],
       [-0.05933099, -0.02927586, -0.0825242 , ...,  0.00904728,
        -0.01464811,  0.1086615 ],
       [ 0.09817139,  0.12248526,  0.01193939, ..., -0.00047453,
        -0.01416449,  0.03580394],
       ...,
       [-0.04307749,  0.02604584, -0.0054444 , ..., -0.05025208,
        -0.02080824, -0.04466284],
       [ 0.05171415, -0.0473899 ,  0.03908065, ..., -0.00095259,
         0.01799553, -0.02662304],
       [-0.07158374,  0.01617636, -0.02732761, ..., -0.00619405,
        -0.02061255,  0.01279712]])

In [11]:
X_lazy.evaluate()

array([[ 1.73922335e-04,  9.05142679e-05,  1.36939406e-04, ...,
        -6.30533325e-05, -4.04816359e-05, -1.44645441e-04],
       [-1.87621062e-04, -9.25784070e-05, -2.60964421e-04, ...,
         2.86100137e-05, -4.63213968e-05,  3.43617843e-04],
       [ 3.10445190e-04,  3.87332416e-04,  3.77556536e-05, ...,
        -1.50059430e-06, -4.47920508e-05,  1.13221996e-04],
       ...,
       [-1.36222991e-04,  8.23641647e-05, -1.72166949e-05, ...,
        -1.58911027e-04, -6.58014228e-05, -1.41236289e-04],
       [ 1.63534492e-04, -1.49860011e-04,  1.23583860e-04, ...,
        -3.01236783e-06,  5.69068506e-05, -8.41894512e-05],
       [-2.26367674e-04,  5.11541508e-05, -8.64175041e-05, ...,
        -1.95873053e-05, -6.51826163e-05,  4.04680450e-05]])

In [12]:
U@S@V.T

array([[ 1.73922335e-04,  9.05142679e-05,  1.36939406e-04, ...,
        -6.30533325e-05, -4.04816359e-05, -1.44645441e-04],
       [-1.87621062e-04, -9.25784070e-05, -2.60964421e-04, ...,
         2.86100137e-05, -4.63213968e-05,  3.43617843e-04],
       [ 3.10445190e-04,  3.87332416e-04,  3.77556536e-05, ...,
        -1.50059430e-06, -4.47920508e-05,  1.13221996e-04],
       ...,
       [-1.36222991e-04,  8.23641647e-05, -1.72166949e-05, ...,
        -1.58911027e-04, -6.58014228e-05, -1.41236289e-04],
       [ 1.63534492e-04, -1.49860011e-04,  1.23583860e-04, ...,
        -3.01236783e-06,  5.69068506e-05, -8.41894512e-05],
       [-2.26367674e-04,  5.11541508e-05, -8.64175041e-05, ...,
        -1.95873053e-05, -6.51826163e-05,  4.04680450e-05]])

In [14]:
lazy(U)

In [15]:
X_lazy2 = lazy(U)@lazy(S)@lazy(V.T)

In [16]:
X_lazy2.evaluate()

array([[ 1.73922335e-04,  9.05142679e-05,  1.36939406e-04, ...,
        -6.30533325e-05, -4.04816359e-05, -1.44645441e-04],
       [-1.87621062e-04, -9.25784070e-05, -2.60964421e-04, ...,
         2.86100137e-05, -4.63213968e-05,  3.43617843e-04],
       [ 3.10445190e-04,  3.87332416e-04,  3.77556536e-05, ...,
        -1.50059430e-06, -4.47920508e-05,  1.13221996e-04],
       ...,
       [-1.36222991e-04,  8.23641647e-05, -1.72166949e-05, ...,
        -1.58911027e-04, -6.58014228e-05, -1.41236289e-04],
       [ 1.63534492e-04, -1.49860011e-04,  1.23583860e-04, ...,
        -3.01236783e-06,  5.69068506e-05, -8.41894512e-05],
       [-2.26367674e-04,  5.11541508e-05, -8.64175041e-05, ...,
        -1.95873053e-05, -6.51826163e-05,  4.04680450e-05]])

In [ ]:
np.eye(10)